발표와 질의응답 : 발표를 누가 하든지 질의응답에서 대답을 잘 못하면 개별 감점이 있습니다.

보강이 필요한데, 영상으로 진행할 예정입니다. 세 시간 정도 앞에서 배운 내용을 요약해주는 형식입니다. 왠만해서는 영상으로 보강이 가능하도록 할 겁니다.

퀴즈는 발표주차 바로 전에 보도록 하겠습니다. 15~20분 정도 분량입니다. 간단히 확인할 수 있는 정도의 문제 -> 조별 편차가 얼마 안나기 때문에 만들어놓은 겁니다.

### SQLite

오픈 소스, 가벼운 유형으로 모든 시스템에서 사용이 가능하다.

데이터 형식의 다양성(스키마 지정), 수평 확장성(데이터베이스 네트워크 간 확장), 데이터 관리의 엄격성에서는 다소 약점이 있고 이런 부분에서는 MySQL이 강점을 가지고 있다.

> 실제 상황과의 괴리 : 실제는 서버에서 돌아가고, 대용량 데이터가 여러 데이터베이스간 교차하여 작동함.

지원하는 데이터 유형

* TEXT 문자열(UTF-8, UTF-16BE, UTF16LE)
* REAL 8바이트 부동 소수점 값. float32
* NULL
* INTEGER 8바이트 정수값. int32
* BLOB 입력형태 그대로 저장(이미지, 오디오, 멀티미디어 파일)

In [1]:
install.packages("DBI")
install.packages("RSQLite")
library(DBI)
library(RSQLite)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [8]:
conn = dbConnect(RSQLite::SQLite(), "CarsDB.db")  ## create DB file of current working space

> 작업공간에 데이터베이스 파일 생성, conn으로 호출

In [6]:
data("mtcars")
mtcars$car_names = rownames(mtcars)
rownames(mtcars) = NULL
head(mtcars)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,car_names
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4,Mazda RX4
2,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4,Mazda RX4 Wag
3,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1,Datsun 710
4,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1,Hornet 4 Drive
5,18.7,8,360,175,3.15,3.440,17.02,0,0,3,2,Hornet Sportabout
6,18.1,6,225,105,2.76,3.460,20.22,1,0,3,1,Valiant


In [9]:
dbWriteTable(conn, "cars_data", mtcars)  ## write relation, table name : cars_data
dbListTables(conn)  ## call DB relation list

[1] "cars_data"

In [10]:
dbDisconnect(conn)  ## disconnecting database

`-` 기초 예제

In [2]:
conn = dbConnect(SQLite(), "CarsDB.db")  ## Connecting DB file

In [12]:
## create dataframe
car <- c('Camaro', 'California', 'Mustang', 'Explorer')
make <- c('Chevrolet','Ferrari','Ford','Ford')
df1 <- data.frame(car,make)

car <- c('Corolla', 'Lancer', 'Sportage', 'XE')
make <- c('Toyota','Mitsubishi','Kia','Jaguar')
df2 = data.frame(car, make)
dfList = list(df1, df2)

for (k in 1:length(dfList)) {
    dbWriteTable(conn, "Cars_and_Makes",
                 dfList[[k]], append = TRUE)
}  ## Write two dataframes into the Cars_and_Makes relation.

> 루프를 이용하여 두 개의 데이터프레임을 추가 삽입. append = TRUE로 지정해야 기존 테이블에 추가할 수 있음.

In [13]:
dbListTables(conn)

dbGetQuery(conn, "SELECT * FROM Cars_and_Makes")  ## SELECT all(*) FROM Cars_and_Makes relation
dbGetQuery(conn, "SELECT * FROM cars_data LIMIT 20")  ## SELECT all(*) columns FROM carsr_data relation.
# maximun 20 tuples
dbGetQuery(conn, "SELECT car_names, hp, cyl FROM cars_data WHERE cyl = 8")

[1] "Cars_and_Makes" "cars_data"

car,make
<chr>,<chr>
Camaro,Chevrolet
California,Ferrari
Mustang,Ford
Explorer,Ford
Corolla,Toyota
Lancer,Mitsubishi
Sportage,Kia
XE,Jaguar


mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,car_names
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,Mazda RX4
21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4,Mazda RX4 Wag
22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1,Datsun 710
21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1,Hornet 4 Drive
18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2,Hornet Sportabout
18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1,Valiant
14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4,Duster 360
24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2,Merc 240D
22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2,Merc 230


car_names,hp,cyl
<chr>,<dbl>,<dbl>
Hornet Sportabout,175,8
Duster 360,245,8
Merc 450SE,180,8
Merc 450SL,180,8
Merc 450SLC,180,8
Cadillac Fleetwood,205,8
Lincoln Continental,215,8
Chrysler Imperial,230,8
Dodge Challenger,150,8


In [26]:
dbGetQuery(conn, "SELECT cyl, AVG(hp) FROM cars_data")

cyl,AVG(hp)
<dbl>,<dbl>
6,146.6875


`"SELECT cyl, AVG(hp) AS 'average_hp', AVG(mpg) AS 'average_mpg' FROM cars_data GROUP BY cyl ORDER BY average_mpg"`

In [20]:
dbGetQuery(conn, "SELECT cyl, AVG(hp) AS 'average_hp', AVG(mpg) AS 'average_mpg' FROM cars_data GROUP BY cyl ORDER BY average_mpg")

cyl,average_hp,average_mpg
<dbl>,<dbl>,<dbl>
8,209.21429,15.10000
6,122.28571,19.74286
4,82.63636,26.66364


SELECT cyl, AVG(hp), AVG(mpg) : hp와 mpg는 평균을 내어 산출

AS 'average_hp', AS 'average_mpg' : 해당 열 이름으로

FROM cars_data

GROUP BY cyl : cyl의 값들을 기준으로 그룹화하여 계산

SELECT car_names, mpg, hp, wt, FROM cars_data WHERE mpg >= ? AND hp >= ? AND wt >= ? : WHERE 구문에 AND, OR 사용 가능

In [12]:
assembleQuery <- function(conn, base, 
 search_parameters)
{ 
parameter_names <- names(search_parameters) 
partial_queries <- "" 
for(k in 1:length(parameter_names))
{ 
filter_k <- paste(parameter_names[k], " >= ? ") 
if(k > 1)
{ filter_k <- paste("AND ", parameter_names[k], " >= ?") } 
partial_queries <- paste(partial_queries, filter_k) 
} 
final_paste <- paste(base, " WHERE", partial_queries) 
print(final_paste) 
values <- unlist(search_parameters, use.names = FALSE) 
result <- dbGetQuery(conn, final_paste, params = values) 
return(result)
}

In [13]:
base = "SELECT car_names, mpg, hp, wt FROM cars_data"
search_parameters = list("mpg" = 16, "hp" = 150, "wt" = 2.1)
result = assembleQuery(conn, base, search_parameters)
result

[1] "SELECT car_names, mpg, hp, wt FROM cars_data  WHERE  mpg  >= ?  AND  hp  >= ? AND  wt  >= ?"


car_names,mpg,hp,wt
<chr>,<dbl>,<dbl>,<dbl>
Hornet Sportabout,18.7,175,3.440
Merc 450SE,16.4,180,4.070
Merc 450SL,17.3,180,3.730
Pontiac Firebird,19.2,175,3.845
Ferrari Dino,19.7,175,2.770


In [15]:
dbExecute(conn, "DELETE FROM cars_data WHERE car_names = 'Mazda RX4'")  ## DB 내용 조작

[1] 1

In [16]:
dbGetQuery(conn, "SELECT * FROM cars_data LIMIT 10")  ## Mazda RX4가 사라짐

mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,car_names
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4,Mazda RX4 Wag
22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1,Datsun 710
21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1,Hornet 4 Drive
18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2,Hornet Sportabout
18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1,Valiant
14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4,Duster 360
24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2,Merc 240D
22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2,Merc 230
19.2,6,167.6,123,3.92,3.440,18.30,1,0,4,4,Merc 280


In [17]:
dbExecute(conn, "INSERT INTO cars_data VALUES (21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,
'Mazda RX4')")

[1] 1

In [19]:
tail(dbGetQuery(conn, "SELECT * FROM cars_data"))  ## Mazda RX4가 사라짐

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,car_names
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
27,30.4,4,95.1,113,3.77,1.513,16.90,1,1,5,2,Lotus Europa
28,15.8,8,351.0,264,4.22,3.170,14.50,0,1,5,4,Ford Pantera L
29,19.7,6,145.0,175,3.62,2.770,15.50,0,1,5,6,Ferrari Dino
30,15.0,8,301.0,335,3.54,3.570,14.60,0,1,5,8,Maserati Bora
31,21.4,4,121.0,109,4.11,2.780,18.60,1,1,4,2,Volvo 142E
32,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,Mazda RX4


> 맨 마지막에 추가됨

`-` 테이블 생성 및 조작

In [56]:
conn = dbConnect(SQLite(), ".temp.db")

dbSendQuery(conn, "CREATE TABLE iris_db (Sepal_Length REAL,\
Sepal_Width REAL, Petal_Length REAL, Petal_Width REAL, Species TEXT)")

<SQLiteResult>
  SQL  CREATE TABLE iris_db (Sepal_Length REAL,
Sepal_Width REAL, Petal_Length REAL, Petal_Width REAL, Species TEXT)
  ROWS Fetched: 0 [complete]
       Changed: 0

> 튜플이 하나도 없어도, 스키마 정보가 있기 때문에 테이블로 저장됨. 직접 스키마를 지정해주고 싶기 때문에 먼저 스키마를 설정하고 그 다음에 데이터를 밀어넣는 것이 좋다.

In [57]:
library(MASS)
data(iris)

In [58]:
colnames(iris) = c("Sepal_Length", "Sepal_Width", "Petal_Length", "Petal_Width", "Species")
dbWriteTable(conn, "iris_db", iris, append = TRUE, rownames = FALSE)  ## 테이블 이름에 맞춰 집어넣음.

Warning message:
“Closing open result set, pending rows”


In [60]:
head(dbGetQuery(conn, "SELECT * FROM iris_db"))

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width,Species
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa


데이터프레임 `iris`를 테이블 이름에 맞춰서 삽입(colnames를 스키마와 일치시켜야 들어감.)

`-` 자료를 직접 삽입(이럴 일은 별로 없을듯.)

In [62]:
dbRemoveTable(conn, "iris_db")
dbSendQuery(conn, "CREATE TABLE iris_db (Sepal_Length REAL,\
Sepal_Width REAL, Petal_Length REAL, Petal_Width REAL, Species TEXT)")

dbSendQuery(conn, "INSERT INTO iris_db \
(Sepal_Length, Sepal_Width, Petal_Length, Petal_Width, Species) \
VALUES (5.1, 3.5, 1.4, 0.2, 'setosa'), \
(5.1, 3.3, 1.5, 0.2, 'setosa')")
dbGetQuery(conn, "SELECT * FROM iris_db")

<SQLiteResult>
  SQL  CREATE TABLE iris_db (Sepal_Length REAL,
Sepal_Width REAL, Petal_Length REAL, Petal_Width REAL, Species TEXT)
  ROWS Fetched: 0 [complete]
       Changed: 0

Warning message:
“Closing open result set, pending rows”


<SQLiteResult>
  SQL  INSERT INTO iris_db 
(Sepal_Length, Sepal_Width, Petal_Length, Petal_Width, Species) 
VALUES (5.1, 3.5, 1.4, 0.2, 'setosa'), 
(5.1, 3.3, 1.5, 0.2, 'setosa')
  ROWS Fetched: 0 [complete]
       Changed: 2

Warning message:
“Closing open result set, pending rows”


Sepal_Length,Sepal_Width,Petal_Length,Petal_Width,Species
<dbl>,<dbl>,<dbl>,<dbl>,<chr>
5.1,3.5,1.4,0.2,setosa
5.1,3.3,1.5,0.2,setosa


In [63]:
#-# 테이블 생성
dbRemoveTable(conn, "iris_db")
dbSendQuery(conn, "CREATE TABLE iris_db (Sepal_Length REAL,\
Sepal_Width REAL, Petal_Length REAL, Petal_Width REAL, Species TEXT)")
colnames(iris) = c("Sepal_Length", "Sepal_Width", "Petal_Length", "Petal_Width", "Species")
dbWriteTable(conn, "iris_db", iris, append = TRUE, rownames = FALSE)  ## 테이블 이름에 맞춰 집어넣음.

#-# 수정
dbSendQuery(conn, "UPDATE iris_db SET Species = 'ss' WHERE Species = 'setosa'")
## 수정 : species가 setosa인 경우 그 이름을 ss로 수정
dbGetQuery(conn, "SELECT * FROM iris_db") |> head()

<SQLiteResult>
  SQL  CREATE TABLE iris_db (Sepal_Length REAL,
Sepal_Width REAL, Petal_Length REAL, Petal_Width REAL, Species TEXT)
  ROWS Fetched: 0 [complete]
       Changed: 0

Warning message:
“Closing open result set, pending rows”


<SQLiteResult>
  SQL  UPDATE iris_db SET Species = 'ss' WHERE Species = 'setosa'
  ROWS Fetched: 0 [complete]
       Changed: 50

Warning message:
“Closing open result set, pending rows”


,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width,Species
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,5.1,3.5,1.4,0.2,ss
2,4.9,3.0,1.4,0.2,ss
3,4.7,3.2,1.3,0.2,ss
4,4.6,3.1,1.5,0.2,ss
5,5.0,3.6,1.4,0.2,ss
6,5.4,3.9,1.7,0.4,ss


`-` 삭제 : Species가 virginica인 튜플들을 삭제함

In [65]:
dbExecute(conn, "DELETE FROM iris_db WHERE Species = 'virginica'")
unique(dbGetQuery(conn, "SELECT * FROM iris_db")$Species)

[1] 0

[1] "ss"         "versicolor"

In [ ]:
dbSendQuery(conn, "UPDATE iris")

`-` 기본키와 외래키의 사용

In [37]:
conn = dbConnect(SQLite(), ".temp.db")

dbSendQuery(conn, "CREATE TABLE iris_db (Sepal_Length REAL,\
Sepal_Width REAL, Petal_Length REAL, Petal_Width REAL, Species TEXT PRIMARY KEY)")
## Species를 기본 키로 지정

<SQLiteResult>
  SQL  CREATE TABLE iris_db (Sepal_Length REAL,
Sepal_Width REAL, Petal_Length REAL, Petal_Width REAL, Species TEXT PRIMARY KEY)
  ROWS Fetched: 0 [complete]
       Changed: 0

In [39]:
dbSendQuery(conn, "INSERT INTO iris_db \
(Sepal_Length, Sepal_Width, Petal_Length, Petal_Width, Species) VALUES \
(5.1, 3.5, 1.4, 0.2, 'setosa')")
dbGetQuery(conn, "SELECT * FROM iris_db")

Warning message:
“Closing open result set, pending rows”


<SQLiteResult>
  SQL  INSERT INTO iris_db 
(Sepal_Length, Sepal_Width, Petal_Length, Petal_Width, Species) VALUES 
(5.1, 3.5, 1.4, 0.2, 'setosa')
  ROWS Fetched: 0 [complete]
       Changed: 1

Warning message:
“Closing open result set, pending rows”


Sepal_Length,Sepal_Width,Petal_Length,Petal_Width,Species
<dbl>,<dbl>,<dbl>,<dbl>,<chr>
5.1,3.5,1.4,0.2,setosa


In [40]:
dbWriteTable(conn, "iris_db", iris, append=TRUE, 
row.names=FALSE)

ERROR: Error: UNIQUE constraint failed: iris_db.Species


> 테이블에 iris 데이터프레임을 집어넣었으나, 기본키 제약을 만족하지 않음.

`-` supplier

In [41]:
dbListTables(conn)

[1] "iris_db"

In [42]:
dbExecute(conn, "PRAGMA foreign_keys = ON")  ## 외래키 사용 활성화
dbExecute(conn, "CREATE TABLE suppliers \
(supplier_id INTEGER PRIMARY KEY, supplier_name TEXT, group_id INTEGER)")

[1] 0

[1] 0

In [48]:
dbExecute(conn, "CREATE TABLE supplier_groups \
(group_id integer PRIMARY KEY, group_name text NOT NULL)")
## 기본키를 group_id로 가지는 공급자 집단 테이블

[1] 0

In [46]:
dbRemoveTable(conn, "suppliers")
dbExecute(conn, "CREATE TABLE suppliers \
(supplier_id INTEGER PRIMARY KEY, supplier_name TEXT NOT NULL, group_id INTEGER,\
FOREIGN KEY (group_id) REFERENCES supplier_groups (group_id) ON UPDATE SET NULL ON DELETE SET NULL)")
## 기본키가 변하면 NULL로 처리

ERROR: Error: no such table: suppliers


In [50]:
dbRemoveTable(conn, "suppliers")
dbExecute(conn, "CREATE TABLE suppliers \
(supplier_id INTEGER PRIMARY KEY, supplier_name TEXT NOT NULL, group_id INTEGER,\
FOREIGN KEY (group_id) REFERENCES supplier_groups (group_id)\
ON UPDATE CASCADE ON DELETE CASCADE)")  ## 기본키 변화를 외래키가 그대로 반영하는 것.

[1] 0

In [ ]:
dbSendQuery(conn, "INSERT INTO supplier_groups (group_name) VALUES ('Domestic'), ('Global'), ('One-Time')")
dbExecute(conn, "INSERT INTO suppliers")

In [ ]:
dbExecute(conn, "CREATE TABLE supplier_groups \
(group_id INTEGER PRIMARY KEY CHECK(group_id <= 100), \
group_name TEXT NOT NULL, \
data_time DEFAULT CURRENT_TIMESTAMP, \
id INTEGER DEFAULT 0, \
iden INTEGER UNIQUE)")

## CHECK 구문을 이용하여 제약조건을 설정
## DEFAULT 설정은 수치형만 가능
## CURRENT_TIMESTAMP는 현재 시각, UNIQUE는 중복을 불허하는 제약조건

### practice

In [85]:
library(DBI)
library(RSQLite)

In [92]:
data("mtcars")
mtcars$car_names = rownames(mtcars) ## 인덱스를 데이터로 따로 받아 새 열로 추가
rownames(mtcars) = NULL  ## 인덱스 초기화
head(mtcars)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,car_names
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4,Mazda RX4
2,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4,Mazda RX4 Wag
3,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1,Datsun 710
4,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1,Hornet 4 Drive
5,18.7,8,360,175,3.15,3.440,17.02,0,0,3,2,Hornet Sportabout
6,18.1,6,225,105,2.76,3.460,20.22,1,0,3,1,Valiant


`-` Creating Database File : `dbConnect()`

In [93]:
conn = dbConnect(RSQLite::SQLite(), "CarsDB.db")  ## create db file

`-` Writing table : `dbWriteTable()` 데이터프레임 삽입에 유리(append)

In [94]:
dbWriteTable(conn, "cars_data", mtcars)  ## Write table(relation)
dbListTables(conn)  ## debuging

[1] "cars_data"

In [95]:
car <- c('Camaro', 'California', 'Mustang', 'Explorer')
make <- c('Chevrolet','Ferrari','Ford','Ford')
df1 <- data.frame(car,make)
car <- c('Corolla', 'Lancer', 'Sportage', 'XE')
make <- c('Toyota','Mitsubishi','Kia','Jaguar')
df2 <- data.frame(car,make)

In [96]:
dfList = list(df1, df2)

for (i in 1:length(dfList)) {
    dbWriteTable(conn, "Cars_and_Makes", dfList[[i]], append = TRUE)  ## write new table
}

dbListTables(conn)

[1] "Cars_and_Makes" "cars_data"

* arguments

> append = TRUE : 기존 테이블에 추가
>
> 작성 후 반드시 dbDisconnect()를 하라고 경고가 발생

`-` Query : `dbGetQuery()` 결과물만 반환할 수 있음.

In [98]:
# dbGetQuery(conn, "SELECT * FROM Cars_and_Makes")
# dbGetQuery(conn, "SELECT * FROM cars_data LIMIT 20")
dbGetQuery(conn, "SELECT car_names, hp, cyl FROM cars_data")

car_names,hp,cyl
<chr>,<dbl>,<dbl>
Mazda RX4,110,6
Mazda RX4 Wag,110,6
Datsun 710,93,4
Hornet 4 Drive,110,6
Hornet Sportabout,175,8
Valiant,105,6
Duster 360,245,8
Merc 240D,62,4
Merc 230,95,4


`-` Query : `dbExecute()` 작업만 수행하고 결과물을 반환하지 않음.

In [99]:
conn = dbConnect(SQLite(), ".temp.db")

In [100]:
## creating table with schema
dbExecute(conn, "CREATE TABLE mytable
          (a INTEGER PRIMARY KEY, 
         b TEXT)")

[1] 0

In [101]:
dbExecute(conn, "INSERT INTO mytable VALUES(1, 'test'), (2, 'test'), (6, 'test')")

[1] 3

In [103]:
dbGetQuery(conn, "SELECT * FROM mytable")

a,b
<int>,<chr>
1,test
2,test
6,test


`-` Query : `dbSendQuery()` 작업을 수행하고 어떤 데이터가 들어갔는지 결과를 반환

In [105]:
dbSendQuery(conn, "INSERT INTO mytable VALUES(4, 'test')")

<SQLiteResult>
  SQL  INSERT INTO mytable VALUES(4, 'test')
  ROWS Fetched: 0 [complete]
       Changed: 1

In [106]:
dbGetQuery(conn, "SELECT * FROM mytable")

Warning message:
“Closing open result set, pending rows”


a,b
<int>,<chr>
1,test
2,test
4,test
6,test


`-` Removing table : `dbRemoveTable()`

In [ ]:
dbRemoveTable(conn, 'test_table')

`-` Query 문법

`"SELECT col_name1 AS col1, AVG(col2) AS average_col_2 FROM table GROUP BY col1 ORDER BY average_col_2"`

> 범용적인 문법, 테이블을 바꾸지 않음. `dbGetQuery()`에서 주로 사용.
>
> `AS`로 열 이름을 변경해서 호출, `AVG()`로 평균을 호출, `GROUP BY`에 해당하는 열 그룹을 기준으로 그룹화하여 통계량 산출, `ORDER BY`에 해당하는 열 그룹을 기준으로 sorting.

`"INSERT INTO table (col1, col2, ...) VALUES (val11, val12, ...), (val21, val22, ...), ..."`

> 테이블에 튜플 삽입, `dbExecute()`, `dbSendQuery()`에서 사용.

`"DELETE FROM table WHERE col_name = value"`

> 테이블의 튜플 제거, `dbExecute()`, `dbSendQuery()`에서 사용.

`"UPDATE table SET col_name = value WHERE col_name = value"`

> 테이블의 튜플 수정, `dbExecute()`, `dbSendQuery()`에서 사용.